In [1]:
import pandas as pd
import numpy as np

from compute_lda import *

/home/raymond/anaconda3/envs/topicModeling/lib/python3.7/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


### Entitiy Matching

In [2]:
shipper_matching = pd.read_csv('../shipper_matching/Enigma_Enigma_6countries.csv')

In [3]:
shipper_matching.shape

(425010, 12)

In [4]:
shipper_matching[(shipper_matching['name_score']>0.9) & (shipper_matching['address_score']>0.6)].shape

(33916, 12)

In [5]:
shipper_matching  = shipper_matching[(shipper_matching['name_score']>0.9) & (shipper_matching['address_score']>0.6)]

In [6]:
def shipper_set(col1,col2):
    return {col1,col2}

In [7]:
shipper_set_vectorize = np.vectorize(shipper_set)
shipper_matching['cl_shipper_set'] = shipper_set_vectorize(shipper_matching['cl_shipper_party_name_left'],shipper_matching['cl_shipper_party_name_right']) 

In [8]:
shipper_matching['cl_shipper_set'].shape

(33916,)

In [9]:
le = preprocessing.LabelEncoder()
le.fit(shipper_matching['cl_shipper_party_name_left'])
shipper_matching['cl_shipper_left_index'] = le.transform(shipper_matching['cl_shipper_party_name_left'].values)
shipper_matching['cl_shipper_riht_index'] = le.transform(shipper_matching['cl_shipper_party_name_right'].values)
#inverse_transform = le.inverse_transform(shipper_matching['cl_shipper_party_name_right'].values)

In [12]:
shipper_matching.head()

,name_score,address_score,identifier_left,identifier_right,shipper_party_name_left,shipper_party_name_right,cl_shipper_party_name_left,cl_shipper_party_name_right,shipper_address_left,shipper_address_right,harmonized_number_left,harmonized_number_right,cl_shipper_set,cl_shipper_left_index,cl_shipper_riht_index
9,0.923077,0.632911,2.018031e+10,2.020000e+12,&TRADITION A/S,TRADITION AS,&TRADITION A/S,TRADITION AS,TRANGRAVSVEJ 24 COPENHAGEN K 1436 DK,KRONPRINSESSEGADE 4 COPENHAGEN 1306 DK,NaN,NaN,"{&TRADITION A/S, TRADITION AS}",0,20057
10,0.961538,0.905660,2.018042e+12,2.018022e+12,"+ CARGA-TRNSITOS, COMERCIO E","+CARGA-TRANSITOS, COMERCIO E",+ CARGATRNSITOS COMERCIO E,+CARGATRANSITOS COMERCIO E,"SERVICOS, LDA. PRACA DUQUE DA TERCEIRA, N24 -3...","SERVICOS, LDA PRACA DUQUE DA TERCEIRA, N-24 -3...",NaN,680292.0,"{+ CARGATRNSITOS COMERCIO E, +CARGATRANSITOS C...",1,2
12,0.909091,0.716763,2.018040e+12,2.018040e+12,1) CERELIA BELGIUM BVBA /,CERELIA BELGIUM BVBA,1 CERELIA BELGIUM BVBA /,CERELIA BELGIUM BVBA,ONDERNEMINGENSTRAAT 5A 8630 VE 4520 VINALMONT ...,ONDERNEMINGENSTRAAT 5A RUE DE VILLERS 36 4520 ...,190532.0,190532.0,"{CERELIA BELGIUM BVBA, 1 CERELIA BELGIUM BVBA /}",4,3441
13,0.983607,0.752475,2.018010e+11,2.020000e+12,1010 PRINTING INTERNATIONAL LTD,1010 PRINTING INTERNATIONA LTD,1010 PRINTING INTERNATIONAL LTD,1010 PRINTING INTERNATIONA LTD,"26/F, 625 KING'S ROAD NORTH POINT HONG KONG ...","26/F, 625 KING'S ROAD HONG KONG NORTH POINT,...",NaN,NaN,"{1010 PRINTING INTERNATIONA LTD, 1010 PRINTING...",6,5
15,0.923077,0.839506,2.018020e+12,2.018040e+12,-18 SAS,-18 SAS,18 SAS,18 SAS,13 CHEMIN DES CHAUDRONNIERS F-94310 ORLY,13 CHEMIN DES CHAUDRONNIERS ORLY 94130 FR,NaN,NaN,"{18 SAS, 18 SAS}",7,8


### Logistics Company

In [13]:
data = read_data()

In [15]:
data.columns

Index(['identifier', 'port_of_unlading', 'estimated_arrival_date',
       'foreign_port_of_lading', 'record_status_indicator', 'place_of_receipt',
       'port_of_destination', 'foreign_port_of_destination',
       'actual_arrival_date', 'consignee_name', 'consignee_address',
       'consignee_contact_name', 'consignee_comm_number_qualifier',
       'consignee_comm_number', 'shipper_party_name', 'shipper_address',
       'shipper_contact_name', 'shipper_comm_number_qualifier',
       'shipper_comm_number', 'description_sequence_number', 'piece_count',
       'description_text', 'harmonized_number', 'harmonized_value',
       'harmonized_weight', 'harmonized_weight_unit'],
      dtype='object')

In [23]:
data.shape

(3825317, 26)

In [28]:
data['shipper_party_name'].count()

3005212

In [31]:
data['shipper_party_name'].str.contains(pat='logistic|logistics|forward|dhl|transport|shipping',case=False).sum()

322561